In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean

# Data Science with Python LN2: 주택실거래가 데이터 분석(1) 이어서...

In [2]:
# Tuple 정의
x=(1,2)
type(x)

tuple

In [3]:
x

(1, 2)

#### list와 tuple의 가장 큰 차이점
- list는 값 추가, 변경 가능
- tuple은 정의 하면 변경 불가능
- tuple 은 append 사용하기 불가능

In [4]:
# 2번째 값을 4로 추가하고 싶다, 라고 하면 에러날 것임
x[2]=4

TypeError: 'tuple' object does not support item assignment

In [5]:
# list에서 값 추가할 때 쓰는 append 도 tuple에선 안됨
# .append(5) = > 마지막에 5 추가해라
x.append(5)

AttributeError: 'tuple' object has no attribute 'append'

#### 반복문 다시

- for

- range *주의!*

- while

In [6]:
animals=['dog','cat','bird']
for i in animals:
    print(i)

dog
cat
bird


In [7]:
# range 사용해서 반복문 쓰기

k=len(animals) #animals 리스트 내의 값의 갯수를 저장한다. length=len
k

3

In [8]:
 # range(k) : 처음부터 k가 3이니까 세번째까지가 아니고 3을 제외한 그 바로 앞, 그니까 두번째까지 0,1,2 이렇게 3개가 나옴
for i in range(k): 
    print(animals[i])

dog
cat
bird


In [9]:
# while 이용해서 loop 실행하기

# i는1, 5보다 작을때까지 loop를 돌린다. 그리고 i를 출력한다. 그리고 1을 더한다

i=1
while i<5:
    print(i)
    i=i+1

1
2
3
4


In [10]:
# 0-9까지 숫자를 2를 매번 곱해서 리스트로 생성하자
# 0,2,4,6,8,10,12,14,16,18 이 리스트로 만들어지면 될 것

for i in range(10):
    x=i*2
    print(x)

0
2
4
6
8
10
12
14
16
18


In [11]:
# 실행된 값을 모아서 리스트로 만들려면.. = Q11

y=[] # 비어있는 리스트 object
for i in range(10):
    x=i*2
    y.append(x) # list append 할 수 있다
print(y)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


#### list comprehension을 사용하면 같은 결과(Q11)를 간단한 코드로 쓸 수 있다.
#### => loop 구문을 list 안에서 정의할 수 있는 것

In [12]:
# list comprehension

y1=[2*x for x in range(10)]
y1

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### 4 API를 이용한 주택실거래가 데이터 수집

 

In [13]:
pip install urllib.request

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib.request (from versions: none)
ERROR: No matching distribution found for urllib.request


In [18]:
pip install requests

In [20]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [21]:
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

In [31]:
# 표에 있는 걸 미리 정의 해줘야 한다
Servicekey="29jTU1S%2B7bDI1Rb2LV0S5D0d3Xn%2Fji9T3k%2FWjsLR%2FyI9O3wEU0atKsr9hj9ATsfIpKgWmV8k8Ll67O%2F73Nyj6w%3D%3D"
month=202401 # 2024년 1월 거래 정보를 사용하고자 한다.
gu_code=11110 # 서울시 종로구 데이터를 사용하겠다
numOfRows=1000 # 최대 1000개의 거래정보를 가져오겠다
url="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD="+str(gu_code)+"&DEAL_YMD="+str(month)+"&numOfRows="+str(numOfRows)+"&serviceKey="+Servicekey

In [32]:
print(url)

http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11110&DEAL_YMD=202401&numOfRows=1000&serviceKey=29jTU1S%2B7bDI1Rb2LV0S5D0d3Xn%2Fji9T3k%2FWjsLR%2FyI9O3wEU0atKsr9hj9ATsfIpKgWmV8k8Ll67O%2F73Nyj6w%3D%3D


In [35]:
# url 불러오기
result=urlopen(url)
house=BeautifulSoup(result,'lxml-xml') # lxml-xml: xml이라는 data를 저장하는 방식 , 외우는 수 밖에 없음!!
house

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     192,000</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 내수동</법정동><아파트>경희궁의아침3단지</아파트><월>1</월><일>13</일><전용면적>149</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>72</지번><지역코드>11110</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     192,000</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 내수동</법정동><아파트>경희궁의아침3단지</아파트><월>1</월><일>13</일><전용면적>149</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>72</지번><지역코드>11110</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>      42,000</거래금액><거래유형>중개거래</거래유형><건축년도>2003</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 익선동</법정동><아파트>현대뜨레비앙</아파트><월>1</월><일>30</일><전용면적>48.54</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>55</지번><지역코드>11110</지역코드><층>2</층><해제사유발생일> </해제사유발생일

In [36]:
# 거래금액, 지역 등 같은 변수 끼리 엑셀 처럼 묶어 보려면...
# item으로 시작해서 끝남 <- 하나의 매물이 item에 들어있음. 

te=house.find_all("item")
te

[<item><거래금액>     192,000</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 내수동</법정동><아파트>경희궁의아침3단지</아파트><월>1</월><일>13</일><전용면적>149</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>72</지번><지역코드>11110</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>,
 <item><거래금액>     192,000</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 내수동</법정동><아파트>경희궁의아침3단지</아파트><월>1</월><일>13</일><전용면적>149</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>72</지번><지역코드>11110</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>,
 <item><거래금액>      42,000</거래금액><거래유형>중개거래</거래유형><건축년도>2003</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 익선동</법정동><아파트>현대뜨레비앙</아파트><월>1</월><일>30</일><전용면적>48.54</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>55</지번><지역코드>11110</지역코드><층>2</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>,
 <item><거래금액>      46,000</거래금액><거래유형>중개거래</거래유형><건축년도>2003</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><

In [38]:
# [<item><거래금액>     192,000</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 내수동</법정동><아파트>경희궁의아침3단지</아파트><월>1</월><일>13</일><전용면적>149</전용면적><중개사소재지>서울 종로구</중개사소재지><지번>72</지번><지역코드>11110</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>
# 이게 하나의 매물

# api url 따라 들어가면 맨 아래에
# <totalCount>18</totalCount>
# 이걸 보고 총 18개 len 인걸 알 수 있음

In [40]:
data=[]
for i in range(len(te)):
    price=te[i].거래금액.string.strip() 
    #.string.strip() 거래금액 19200을 숫자 말고 문자로 가져와라
    apt_name=te[i].아파트.string.strip()
    total=[price,apt_name] # Price랑 apt_name 묶어서 리스트로 나타내자
    data.append(total)
print(data)

[['192,000', '경희궁의아침3단지'], ['192,000', '경희궁의아침3단지'], ['42,000', '현대뜨레비앙'], ['46,000', '리치캐슬아파트'], ['57,400', '송림아마레스아파트'], ['57,500', '창신쌍용2'], ['13,500', '삼전솔하임4차'], ['95,000', '종로센트레빌'], ['101,300', '종로청계힐스테이트'], ['80,000', '종로청계힐스테이트'], ['200,000', '경희궁자이(3단지)'], ['195,000', '경희궁자이(2단지)'], ['200,000', '경희궁자이(2단지)'], ['156,000', '경희궁자이(2단지)'], ['135,000', '롯데캐슬로잔'], ['70,500', '일성빌라트'], ['39,000', '무악다온채'], ['101,000', '인왕산아이파크']]


In [44]:
# 동 이름도 넣어보자
data1=[]
for i in range(len(te)):
    price=te[i].거래금액.string.strip() 
    #.string.strip() 거래금액 19200을 숫자 말고 문자로 가져와라
    apt_name=te[i].아파트.string.strip()
    dong_name=te[i].법정동.string.strip()
    
    total1=[price,apt_name,dong_name] # Price랑 apt_name 묶어서 리스트로 나타내자
    data1.append(total1)
print(data1) #list type의 데이터로 저장

[['192,000', '경희궁의아침3단지', '내수동'], ['192,000', '경희궁의아침3단지', '내수동'], ['42,000', '현대뜨레비앙', '익선동'], ['46,000', '리치캐슬아파트', '명륜1가'], ['57,400', '송림아마레스아파트', '명륜1가'], ['57,500', '창신쌍용2', '창신동'], ['13,500', '삼전솔하임4차', '숭인동'], ['95,000', '종로센트레빌', '숭인동'], ['101,300', '종로청계힐스테이트', '숭인동'], ['80,000', '종로청계힐스테이트', '숭인동'], ['200,000', '경희궁자이(3단지)', '평동'], ['195,000', '경희궁자이(2단지)', '홍파동'], ['200,000', '경희궁자이(2단지)', '홍파동'], ['156,000', '경희궁자이(2단지)', '홍파동'], ['135,000', '롯데캐슬로잔', '평창동'], ['70,500', '일성빌라트', '평창동'], ['39,000', '무악다온채', '무악동'], ['101,000', '인왕산아이파크', '무악동']]


#### 마지막으로 저장된 list "data"를 excel 또는 csv 파일로 내보내보자

#### csv 파일로 저장해보자
- 1단계: list를 데이터 프레임으로 생성한다. pandas의 data frame으로 변환한다
- 2단계: panadas data frame을 csv 파일로 내보낸다


In [49]:
# data frame 이름을 temp로 지정한다
#coulmns는 데이터 프레임에서 각 칼럼의 이름을 지정한다

In [51]:
import pandas as pd

In [53]:
temp=pd.DataFrame(data1, columns=['price', 'apt_name', 'dong_name'])
temp


,price,apt_name,dong_name
0,"192,000",경희궁의아침3단지,내수동
1,"192,000",경희궁의아침3단지,내수동
2,"42,000",현대뜨레비앙,익선동
3,"46,000",리치캐슬아파트,명륜1가
4,"57,400",송림아마레스아파트,명륜1가
5,"57,500",창신쌍용2,창신동
6,"13,500",삼전솔하임4차,숭인동
7,"95,000",종로센트레빌,숭인동
8,"101,300",종로청계힐스테이트,숭인동
9,"80,000",종로청계힐스테이트,숭인동


In [54]:
#csv 파일로 내보내기
temp.to_csv("c:/temp/house_price100.csv")